In [8]:
import cvzone
import cv2
from cvzone.HandTrackingModule import HandDetector
import numpy as np
import google.generativeai as genai
from PIL import Image
import streamlit as st
 
 

In [9]:
import speech_recognition as sr

import pyttsx3


In [14]:
st.set_page_config(layout="wide")
st.image('D:\school projects\idk\cv proj\main.py\pepeCool.png')
 
col1, col2 = st.columns([3,2])
with col1:
   run = st.checkbox('Run', value=True)
   FRAME_WINDOW = st.image([])
 
with col2:
   st.title("Answer")
   output_text_area = st.subheader("")

2024-10-22 17:59:54.107 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-22 17:59:54.109 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-22 17:59:54.113 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-22 17:59:54.116 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-22 17:59:54.119 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-22 17:59:54.121 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-22 17:59:54.123 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-22 17:59:54.125 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [14]:
import pyttsx3



import customtkinter as ctk

def show_modern_popup(message="Hello!", title="Message", width=800, height=400):
    # Configure the appearance
    ctk.set_appearance_mode("dark")
    ctk.set_default_color_theme("dark-blue")
    
    # Create and configure the window
    popup = ctk.CTk()
    popup.title(title)
    popup.geometry(f"{width}x{height}")
    
    # Center the window on screen
    screen_width = popup.winfo_screenwidth()
    screen_height = popup.winfo_screenheight()
    center_x = int(screen_width/2 - width/2)
    center_y = int(screen_height/2 - height/2)
    popup.geometry(f'+{center_x}+{center_y}')
    
    # Add message label
    label = ctk.CTkLabel(
        popup,
        text=message,
        font=("Helvetica", 14),
        wraplength=width-40  # Wrap text if too long
    )
    label.pack(pady=20, padx=20, expand=True)
    
    # Add OK button
    button = ctk.CTkButton(
        popup,
        text="OK",
        command=popup.destroy,
        width=100,
        height=32,
        font=("Helvetica", 12)
    )
    button.pack(pady=(0, 20))
    
    # Make window modal (user must interact with it)
    popup.transient()
    popup.grab_set()
    
    # Start the popup
    popup.mainloop()

# Example usage
    
# st.set_page_config(layout="wide")
# st.image('D:\school projects\idk\cv proj\main.py\pepeCool.png')
 
# col1, col2 = st.columns([3,2])
# with col1:
#    run = st.checkbox('Run', value=True)
#    FRAME_WINDOW = st.image([])
 
# with col2:
#    st.title("Answer")
#    output_text_area = st.subheader("")
 
 
genai.configure(api_key="XXX")
model = genai.GenerativeModel('gemini-1.5-flash-002')
 
# Initialize the webcam to capture video
# The '2' indicates the third camera connected to your computer; '0' would usually refer to the built-in camera
cap = cv2.VideoCapture(0)
cap.set(3,1280)
cap.set(4,720)
 
# Initialize the HandDetector class with the given parameters
detector = HandDetector(staticMode=False, maxHands=1, modelComplexity=1, detectionCon=0.7, minTrackCon=0.5)
 
 
def getHandInfo(img):
    # Find hands in the current frame
    # The 'draw' parameter draws landmarks and hand outlines on the image if set to True
    # The 'flipType' parameter flips the image, making it easier for some detections
    hands, img = detector.findHands(img, draw=False, flipType=True)
 
    # Check if any hands are detected
    if hands:
        # Information for the first hand detected
        hand = hands[0]  # Get the first hand detected
        lmList = hand["lmList"]  # List of 21 landmarks for the first hand
        # Count the number of fingers up for the first hand
        fingers = detector.fingersUp(hand)
        #print(fingers)
        return fingers, lmList
    else:
        return None
 
def draw(info,prev_pos,canvas):
    fingers, lmList = info
    current_pos= None
    if fingers == [1, 1, 0, 0, 0] or fingers ==[0,1,0,0,0]:
        current_pos = lmList[8][0:2]
        if prev_pos is None: prev_pos = current_pos
        cv2.line(canvas,current_pos,prev_pos,(255,0,255),10)    
    elif fingers == [0, 0, 0, 0, 1]:
        canvas = np.zeros_like(img)
 
    return current_pos, canvas
 
def sendToAI(model, canvas, fingers):
    if fingers == [1, 1, 1, 0, 0]:
        pil_image = Image.fromarray(canvas)
        text_to_speech("ask your question")
        user_query = speech_to_text()
        if user_query:
            text_to_speech("i am solving the problem")
            response = model.generate_content([f"Answer this maths question: {user_query}", pil_image])
            return response.text
        else:
            return "Could not understand the query."
 


def speech_to_text():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        audio = recognizer.listen(source)
        try:
            text = recognizer.recognize_google(audio)
            print("You said: " + text)
            return text
        except sr.UnknownValueError:
            print("Google Speech Recognition could not understand audio")
            return None
        except sr.RequestError as e:
            print("Could not request results from Google Speech Recognition service; {0}".format(e))
            return None
 
def text_to_speech(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

prev_pos = None
canvas=None
image_combined = None
output_text= ""
prev_output = None
# Continuously get frames from the webcam
while True:
    # Capture each frame from the webcam
    # 'success' will be True if the frame is successfully captured, 'img' will contain the frame
    success, img = cap.read()
    img = cv2.flip(img, 1)
 
    if canvas is None:
        canvas = np.zeros_like(img)
 
 
    info = getHandInfo(img)
    if info:
        fingers, lmList = info
        prev_pos,canvas = draw(info, prev_pos,canvas)
        output_text = sendToAI(model,canvas,fingers)
 
    image_combined= cv2.addWeighted(img,0.7,canvas,0.3,0)
    #FRAME_WINDOW.image(image_combined,channels="BGR")
 
    if output_text != prev_output:
        if output_text != None and output_text != "":
            #output_text_area.text(output_text)
            print(output_text)
            show_modern_popup(
            message=output_text,
            title="Answer"
        )
            #text_to_speech(output_text)
        prev_output = output_text
    
     # Display the image in a window
   #cv2.imshow("Image", img)
    #cv2.imshow("Canvas", canvas)
    cv2.imshow("image_combined", image_combined)
 
 
    # Keep the window open and update it for each frame; wait for 1 millisecond between frames
    if cv2.waitKey(1) & 0xFF == ord('q'):
        #   pass
        break
cap.release()
cv2.destroyAllWindows()

Listening...
You said: find the area of this rectangle
Here's how to solve this:

**1. Identify the dimensions:**

The image shows a rectangle with a width of 5 and a height of 10.

**2. Calculate the area:**

The area of a rectangle is found by multiplying its width and height.  Therefore:

Area = width × height = 5 × 10 = 50

**Answer:** The area of the rectangle is 50 square units.
Listening...
You said: find the area of the circle
Here's how to solve this:

**1. Identify the radius:**

The image shows a line segment within the circle labeled "5".  This line segment represents the radius (r) of the circle.  Therefore, r = 5 units.

**2. Use the area formula:**

The area (A) of a circle is given by the formula: A = πr²

**3. Calculate the area:**

Substitute the radius (r = 5) into the formula:

A = π * 5² = 25π

**4. Approximate the answer:**

Using the approximation π ≈ 3.14159:

A ≈ 25 * 3.14159 ≈ 78.53975 square units

Therefore, the area of the circle is approximately 78.54 squa

KeyboardInterrupt: 

: 

In [8]:
!streamlit run C:\Users\ADMIN\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py

^C


In [4]:
import customtkinter as ctk

def show_modern_popup(message="Hello!", title="Message", width=400, height=200):
    # Configure the appearance
    ctk.set_appearance_mode("dark")
    ctk.set_default_color_theme("dark-blue")
    
    # Create and configure the window
    popup = ctk.CTk()
    popup.title(title)
    popup.geometry(f"{width}x{height}")
    
    # Center the window on screen
    screen_width = popup.winfo_screenwidth()
    screen_height = popup.winfo_screenheight()
    center_x = int(screen_width/2 - width/2)
    center_y = int(screen_height/2 - height/2)
    popup.geometry(f'+{center_x}+{center_y}')
    
    # Add message label
    label = ctk.CTkLabel(
        popup,
        text=message,
        font=("Helvetica", 14),
        wraplength=width-40  # Wrap text if too long
    )
    label.pack(pady=20, padx=20, expand=True)
    
    # Add OK button
    button = ctk.CTkButton(
        popup,
        text="OK",
        command=popup.destroy,
        width=100,
        height=32,
        font=("Helvetica", 12)
    )
    button.pack(pady=(0, 20))
    
    # Make window modal (user must interact with it)
    popup.transient()
    popup.grab_set()
    
    # Start the popup
    popup.mainloop()

# Example usage
if __name__ == "__main__":
    show_modern_popup(
        message="This is a modern dark-themed popup with a clean design!",
        title="Modern Popup"
    )

In [11]:
 
def speech_to_text():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        audio = recognizer.listen(source)
        try:
            text = recognizer.recognize_google(audio)
            print("You said: " + text)
            return text
        except sr.UnknownValueError:
            print("Google Speech Recognition could not understand audio")
            return None
        except sr.RequestError as e:
            print("Could not request results from Google Speech Recognition service; {0}".format(e))
            return None

In [ ]:
import tkinter as tk
from tkinter import messagebox

# Function to show the pop-up
def show_popup():
    messagebox.showinfo("Popup", "This is a pop-up with text!")

# Create the main window
root = tk.Tk()
root.withdraw()  # Hide the root window

# Show the pop-up
show_popup()

# Keep the window open
root.mainloop()
